In [ ]:
# Mount data from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# go to the folder with data
# need to be changed to the specific path of the user
cd 'drive/MyDrive/CS 769/Minyi Trial/data augmentation'

/content/drive/MyDrive/CS 769/Minyi Trial/data augmentation


In [ ]:
# install necessary packages
pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd 
import random
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# check if cuda is available
torch.cuda.is_available()

True

In [ ]:
# output cuda version
import torch
import transformers
torch.__version__, transformers.__version__

('2.0.0+cu118', '4.28.1')

In [ ]:
# install packages for data augmentation
pip install requests nlpaug sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 63.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=f6bf18239640e50361a0809ed479075e4065b1068aed07fc6702a19f93bb3f96
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [ ]:
# get class statistics of training data
# get all classes

file_path = 'techniques_list_task3.txt'
CLASSES = []
with open(file_path, 'r', encoding='utf8') as f:
  for label in f.readlines():
    label = label.strip()
    if label:
      CLASSES.append(label)

CLASSES = {key: 0 for key in CLASSES}

print(CLASSES)

{'Appeal to authority': 0, 'Appeal to fear/prejudice': 0, 'Black-and-white Fallacy/Dictatorship': 0, 'Causal Oversimplification': 0, 'Doubt': 0, 'Exaggeration/Minimisation': 0, 'Flag-waving': 0, 'Glittering generalities (Virtue)': 0, 'Loaded Language': 0, "Misrepresentation of Someone's Position (Straw Man)": 0, 'Name calling/Labeling': 0, 'Obfuscation, Intentional vagueness, Confusion': 0, 'Presenting Irrelevant Data (Red Herring)': 0, 'Reductio ad hitlerum': 0, 'Repetition': 0, 'Slogans': 0, 'Smears': 0, 'Thought-terminating cliché': 0, 'Whataboutism': 0, 'Bandwagon': 0, 'Transfer': 0, 'Appeal to (Strong) Emotions': 0}


In [ ]:
# load json file
import itertools
import json
import os

train_file = 'training_set_task3.txt'
with open(train_file, 'r', encoding='utf8') as f:
    train_data = json.load(f)
  
print(len(train_data))

val_file = 'dev_set_task3_labeled.txt'
with open(val_file, 'r', encoding='utf8') as f:
  val_data = json.load(f)

print(len(val_data))

687
63


In [ ]:
# the training data and validation data are combined and divided into six folds
# the first fold is used for validation and the remaining five folds are used for training
# So we load folds.json to put all the training data id as the key of the dictionary
# This dictionary "aug_dict" will be used in the data augmentation step to make sure 
# only the training data is augmentated. 

fold_file = 'folds.json'
with open(fold_file, 'r', encoding='utf8') as f:
  folds = json.load(f)

print(folds)

aug_array = []
for id in range(1, 6):
  aug_array = aug_array + folds[str(id)]
print(len(aug_array))
aug_dict = {key: 1 for key in aug_array}
print(aug_dict)


{'0': ['14', '188', '501_batch_2', '251_batch_2', '441_batch_2', '611_batch_2', '268_batch_2', '232_batch_2', '604_batch_2', '464_batch_2', '103_batch_2', '631_batch_2', '836_batch_2', '595_batch_2', '118', '143_batch_2', '341_batch_2', '151', '330_batch_2', '665_batch_2', '422_batch_2', '346_batch_2', '426_batch_2', '61_batch_2', '449_batch_2', '602_batch_2', '278_batch_2', '636_batch_2', '133_batch_2', '66_batch_2', '6_batch_2', '31_batch_2', '34_batch_2', '396_batch_2', '376_batch_2', '275_batch_2', '175', '536_batch_2', '201_batch_2', '114', '453_batch_2', '800_batch_2', '391_batch_2', '145_batch_2', '235_batch_2', '360_batch_2', '525_batch_2', '182_batch_2', '793_batch_2', '370_batch_2', '382_batch_2', '17_batch_2', '155_batch_2', '459_batch_2', '786_batch_2', '211_batch_2', '374_batch_2', '24_batch_2', '136_batch_2', '46_batch_2', '210_batch_2', '1', '369_batch_2', '539_batch_2', '239_batch_2', '660_batch_2', '364_batch_2', '583_batch_2', '892_batch_2', '178', '73_batch_2', '85_b

In [ ]:
# Back translation for data augmentation

import random
import nlpaug.augmenter.word as naw

random.seed(42)
def generate_augmented_data(data, augmentation_probability=0.5):
    augmented_data = []
    augmented_id = []
    
    # specify augmenter
    back_translation_aug = naw.BackTranslationAug(
        from_model_name='facebook/wmt19-en-de', 
        to_model_name='facebook/wmt19-de-en',
        device = 'cuda'
    )

    for item in data:
        # Check if the random number is less than augmentation_probability
        if random.random() < augmentation_probability and item['id'] in aug_dict:
            # Perform text augmentation
            #print(item["text"])

            text_parts = item["text"].split('\n\n')

            # Perform text augmentation on each part
            augmented_text_parts = [back_translation_aug.augment(part)[0] if len(back_translation_aug.augment(part)) > 0 else part for part in text_parts]

            # Join the augmented text parts back together using '\n\n'
            augmented_text = '\n\n'.join(augmented_text_parts) + '\n'
            #print(augmented_text)

            # Create a new entry with the augmented text, same labels, and updated ID
            new_entry = {
                "id": f"{item['id']}_aug",
                "labels": item["labels"],
                "text": augmented_text,
                "image": item["image"],
            }

            # Append the new entry to the augmented_data list
            augmented_data.append(new_entry)
            augmented_id.append(new_entry['id'])

    return augmented_data, augmented_id

# Generate augmented data from the original data with a 0.5 probability
augmented_train, augmented_train_id = generate_augmented_data(train_data, 0.5)
# Print the first augmented entry (if available)
print(len(augmented_train))
# save data
combined_train = train_data + augmented_train
print(len(combined_train))
# Save the combined data to a new JSON file
with open("training_aug_set_task3.txt", "w") as outfile:
    json.dump(combined_train, outfile, indent=4)

# Generate augmented data from the original data with a 0.5 probability
augmented_valid, augmented_valid_id = generate_augmented_data(val_data, 0.5)
# Print the first augmented entry (if available)
print(len(augmented_valid))
# save data
combined_valid = val_data + augmented_valid
print(len(combined_valid))
# Save the combined data to a new JSON file
with open("valid_aug_set_task3.txt", "w") as outfile:
    json.dump(combined_valid, outfile, indent=4)

# put the new aug id in the json folder
new_folds = folds.copy()
new_folds["6"] = augmented_train_id + augmented_valid_id
with open("folds_aug.json", "w") as outfile:
  json.dump(new_folds, outfile)


264
951
23
86


In [ ]:
new_folds["6"]

['189_aug',
 '96_aug',
 '145_aug',
 '153_aug',
 '130_aug',
 '168_aug',
 '106_aug',
 '66_aug',
 '127_aug',
 '131_aug',
 '182_aug',
 '132_aug',
 '150_aug',
 '152_aug',
 '193_aug',
 '161_aug',
 '105_aug',
 '177_aug',
 '192_aug',
 '160_aug',
 '181_aug',
 '108_aug',
 '17_aug',
 '133_aug',
 '12_aug',
 '3_aug',
 '137_aug',
 '42_aug',
 '199_aug',
 '53_aug',
 '49_aug',
 '184_aug',
 '149_aug',
 '148_aug',
 '186_aug',
 '5_aug',
 '188_batch_2_aug',
 '107_batch_2_aug',
 '21_batch_2_aug',
 '86_batch_2_aug',
 '70_batch_2_aug',
 '179_batch_2_aug',
 '126_batch_2_aug',
 '26_batch_2_aug',
 '23_batch_2_aug',
 '192_batch_2_aug',
 '170_batch_2_aug',
 '186_batch_2_aug',
 '104_batch_2_aug',
 '63_batch_2_aug',
 '148_batch_2_aug',
 '177_batch_2_aug',
 '45_batch_2_aug',
 '151_batch_2_aug',
 '55_batch_2_aug',
 '29_batch_2_aug',
 '100_batch_2_aug',
 '88_batch_2_aug',
 '171_batch_2_aug',
 '36_batch_2_aug',
 '130_batch_2_aug',
 '75_batch_2_aug',
 '41_batch_2_aug',
 '200_batch_2_aug',
 '152_batch_2_aug',
 '203_batch_

In [ ]:
# Random words insertion for data augmentation

import random
import nlpaug.augmenter.word as naw

random.seed(42)
def generate_augmented_data_sub(data, augmentation_probability=0.5):
    augmented_data = []
    augmented_id = []
    
    aug = naw.ContextualWordEmbsAug(
        model_path='bert-base-uncased', action="substitute", device='cuda')

    for item in data:
        # Check if the random number is less than augmentation_probability
        if random.random() < augmentation_probability and item['id'] in aug_dict:
            # Perform text augmentation
            #print(item["text"])

            text_parts = item["text"].split('\n\n')

            # Perform text augmentation on each part
            augmented_text_parts = [aug.augment(part)[0] if len(aug.augment(part)) > 0 else part for part in text_parts]

            # Join the augmented text parts back together using '\n\n'
            augmented_text = '\n\n'.join(augmented_text_parts) + '\n'
            #print(augmented_text)

            # Create a new entry with the augmented text, same labels, and updated ID
            new_entry = {
                "id": f"{item['id']}_sub",
                "labels": item["labels"],
                "text": augmented_text,
                "image": item["image"],
            }

            # Append the new entry to the augmented_data list
            augmented_data.append(new_entry)
            augmented_id.append(new_entry['id'])

    return augmented_data, augmented_id

# Generate augmented data from the original data with a 0.5 probability
augmented_train, augmented_train_id = generate_augmented_data_sub(train_data, 0.5)
# Print the first augmented entry (if available)
print(len(augmented_train))
# save data
combined_train = train_data + augmented_train
print(len(combined_train))
# Save the combined data to a new JSON file
with open("training_sub_set_task3.txt", "w") as outfile:
    json.dump(combined_train, outfile, indent=4)

# Generate augmented data from the original data with a 0.5 probability
augmented_valid, augmented_valid_id = generate_augmented_data_sub(val_data, 0.5)
# Print the first augmented entry (if available)
print(len(augmented_valid))
# save data
combined_valid = val_data + augmented_valid
print(len(combined_valid))
# Save the combined data to a new JSON file
with open("valid_sub_set_task3.txt", "w") as outfile:
    json.dump(combined_valid, outfile, indent=4)

# put the new aug id in the json folder
new_folds = folds.copy()
new_folds["6"] = augmented_train_id + augmented_valid_id
with open("folds_sub.json", "w") as outfile:
  json.dump(new_folds, outfile)


275
962
27
90


In [ ]:
# Random word substituion for data augmentation

import random
import nlpaug.augmenter.word as naw

random.seed(42)
def generate_augmented_data_insert(data, augmentation_probability=0.5):
    augmented_data = []
    augmented_id = []
    
    aug = naw.ContextualWordEmbsAug(
        model_path='bert-base-uncased', action="insert", device='cuda')

    for item in data:
        # Check if the random number is less than augmentation_probability
        if random.random() < augmentation_probability and item['id'] in aug_dict:
            # Perform text augmentation
            #print(item["text"])

            text_parts = item["text"].split('\n\n')

            # Perform text augmentation on each part
            augmented_text_parts = [aug.augment(part)[0] if len(aug.augment(part)) > 0 else part for part in text_parts]

            # Join the augmented text parts back together using '\n\n'
            augmented_text = '\n\n'.join(augmented_text_parts) + '\n'
            #print(augmented_text)

            # Create a new entry with the augmented text, same labels, and updated ID
            new_entry = {
                "id": f"{item['id']}_insert",
                "labels": item["labels"],
                "text": augmented_text,
                "image": item["image"],
            }

            # Append the new entry to the augmented_data list
            augmented_data.append(new_entry)
            augmented_id.append(new_entry['id'])

    return augmented_data, augmented_id

# Generate augmented data from the original data with a 0.5 probability
augmented_train, augmented_train_id = generate_augmented_data_insert(train_data, 0.5)
# Print the first augmented entry (if available)
print(len(augmented_train))
# save data
combined_train = train_data + augmented_train
print(len(combined_train))
# Save the combined data to a new JSON file
with open("training_insert_set_task3.txt", "w") as outfile:
    json.dump(combined_train, outfile, indent=4)

# Generate augmented data from the original data with a 0.5 probability
augmented_valid, augmented_valid_id = generate_augmented_data_sub(val_data, 0.5)
# Print the first augmented entry (if available)
print(len(augmented_valid))
# save data
combined_valid = val_data + augmented_valid
print(len(combined_valid))
# Save the combined data to a new JSON file
with open("valid_insert_set_task3.txt", "w") as outfile:
    json.dump(combined_valid, outfile, indent=4)

# put the new aug id in the json folder
new_folds = folds.copy()
new_folds["6"] = augmented_train_id + augmented_valid_id
with open("folds_insert.json", "w") as outfile:
  json.dump(new_folds, outfile)


275
962
27
90
